# r-3366

Taken on March 19th, Krypton run, Krypton trigger with "November 20162 conditions

In [ ]:
import datetime

In [ ]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math


In [ ]:
from invisible_cities.database import load_db
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.reco.peak_functions_c as cpf
import invisible_cities.reco.peak_functions as pf
import invisible_cities.reco.pmaps_functions as pmf

In [ ]:
from invisible_cities.reco.pmap_io import pmap_writer, S12, S2Si

from   invisible_cities.core.configure import configure, print_configuration

from   invisible_cities.core.system_of_units_c import units
from   invisible_cities.cities.base_cities import PmapCity, SensorParams
from   invisible_cities.cities.base_cities import S12Params as S12P

In [ ]:
from invisible_cities.core.core_functions import define_window
from invisible_cities.core.core_functions import lrange
from collections import namedtuple

In [ ]:
from invisible_cities.reco.params import S12Params, ThresholdParams, CalibratedSum, PMaps,\
                                         SensorParams, CalibParams, PmapParams, CalibratedPMT, S1PMaps

In [ ]:
from matplotlib.figure import Figure

In [ ]:
%load_ext Cython

In [ ]:
from event_pmaps import pmt_calib_vectors, compare_S1

## Files

In [ ]:
RWF_file = os.path.join(os.environ['IC_DATA'],'LSC/3366/dst_waves_f0_3366.root.h5')
RWF_file = os.path.join(os.environ['IC_DATA'],'LSC/3373/dst_waves_f0_3373.root.h5')

In [ ]:
h5rwf = False

In [ ]:
if h5rwf:
    h5rwf.close()

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
h5rwf

### pmtrwf and sipmrwf vectors

To get vectors use **get_vectors(file)** in tbl_functions

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### Access to data base and definition of units

In [ ]:
DataPMT = load_db.DataPMT()
adc_to_pes = abs(DataPMT.adc_to_pes.values)
coeff_c = abs(DataPMT.coeff_c.values)
coeff_blr = abs(DataPMT.coeff_blr.values)
DataSiPM = load_db.DataSiPM()
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

In [ ]:
P = pmt_calib_vectors()

In [ ]:
P

### RWF

In [ ]:
event=2
RWF = pmtrwf[event]

In [ ]:
wfm.plot_pmt_waveforms(RWF, zoom=False, window_size=10000)

In [ ]:
CWF = blr.deconv_pmt(RWF, P.coeff_c, P.coeff_blr, n_baseline=48000, thr_trigger=5)

In [ ]:
wfm.plot_pmt_waveforms(CWF, zoom=True, window_size=10000)

In [ ]:
CAL_PMT, CAL_PMT_MAU    =    cpf.calibrated_pmt_mau(
                               CWF,
                               P.adc_to_pes,
                               pmt_active =P.pmt_active,
                               n_MAU = 100,
                               thr_MAU =  3)

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(CAL_PMT,
                             pmt_active = P.pmt_active,
                             t_min      = 0,
                             t_max      = 1200,
                             signal_min =    0,
                             signal_max =  10)

In [ ]:
csum, csum_mau = cpf.calibrated_pmt_sum(CWF,P.adc_to_pes,P.pmt_active)

In [ ]:
mpl.plot_signal_vs_time_mus(csum, t_min=0, t_max=1300, signal_min=-5, signal_max=60, label='CSUM')

In [ ]:
mpl.plot_pmt_signals_vs_time_mus(CAL_PMT,
                             pmt_active = P.pmt_active,
                             t_min      = 0,
                             t_max      = 1300,
                             signal_min =    -2,
                             signal_max =  10)

In [ ]:
mpl.plot_calibrated_sum_in_mus(CalibratedSum(csum=csum, csum_mau=csum_mau), 
                               tmin=331, tmax=333, signal_min=-2, signal_max=20, 
                               csum=True, csum_mau=True)

In [ ]:
mpl.plot_signal_vs_time_mus(csum, t_min=640, t_max=670, signal_min=-2, signal_max=60, label='CSUM')

### s12 searches

In [ ]:
s1par = S12Params(tmin=0*units.mus, tmax=640*units.mus, lmin=6, lmax=40, stride=10, rebin=False)
s1par_PMT = S12Params(tmin=331*units.mus, tmax=333*units.mus, lmin=3, lmax=30, stride=4, rebin=False)

In [ ]:
s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.5)
S1 =  cpf.find_S12(s1_ene, s1_indx, **s1par._asdict())

In [ ]:
S1

In [ ]:
pmf.scan_s12(S1)

In [ ]:
plot_s12(S1)

In [ ]:
PMT_S1 = {}
    
for pmt in P.pmt_active:
    s1_ene, s1_indx = cpf.wfzs(CAL_PMT_MAU[pmt], threshold=0.1)
    PMT_S1[pmt] = cpf.find_S12(s1_ene, s1_indx, **s1par_PMT._asdict())

In [ ]:
PMT_S1

In [ ]:
compare_S1(S1, PMT_S1)

In [ ]:
s2_par = S12Params(tmin=640*units.mus, tmax=800*units.mus, stride=40, lmin=80, lmax=20000, rebin=True)

In [ ]:
s2_ene, s2_indx = cpf.wfzs(csum, threshold=0.5)
S2              = cpf.find_S12(s2_ene, s2_indx, **s2_par._asdict())

In [ ]:
S2

In [ ]:
def plot_s12(S12):
    """Plot the peaks of input S12.

    S12 is a dictionary
    S12[i] for i in keys() are the S12 peaks
    """
    plt.figure(figsize=(10, 10))
    mpl.set_plot_labels(xlabel = "t (mus)",
                    ylabel = "S12 (pes)")
    xy = len(S12)
    if xy == 1:
        t = S12[0][0]
        E = S12[0][1]
        ax1 = plt.subplot(1, 1, 1)
        plt.plot(t, E)
    else:
        x = 3
        y = xy/x
        if y % xy != 0:
            y = int(xy/x) + 1
        for i in S12.keys():
            ax1 =  plt.subplot(x, y, i+1)
            t = S12[i][0]
            E = S12[i][1]
            plt.plot(t, E)


In [ ]:
pmf.scan_s12(S2)

In [ ]:
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=3.5*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
S2Si = pf.sipm_s2_dict(SIPM, S2, thr=5*units.pes)

In [ ]:
S2Si

In [ ]:
def plot_s2si_map(S2Si, cmap='Dark2'):
        """Plot a map of the energies of S2Si objects."""
        from   invisible_cities.core.mpl_functions import circles

        DataSensor = load_db.DataSiPM()
        radius = 3
        xs = DataSensor.X.values
        ys = DataSensor.Y.values
        r = np.ones(len(xs)) * radius
        col = np.zeros(len(xs))
        for sipm in S2Si.values():
            for nsipm, E in sipm.items():
                ene = np.sum(E)
                col[nsipm] = ene
        plt.figure(figsize=(8, 8))
        plt.subplot(aspect="equal")
        circles(xs, ys, r, c=col, alpha=0.5, ec="none", cmap=cmap)
        plt.colorbar()

        plt.xlim(-198, 198)
        plt.ylim(-198, 198)

In [ ]:
plot_s2si_map(S2Si)

In [ ]:
def event_pmaps(P,
                pmtrwf, sipmrwf,
                s1par, s2par, thr,
                s12f,
                deconv_n_baseline  = 48000,
                deconv_thr_trigger =     5,
                s1_match_pmt       = 1 * units.mus,
                event=0):
    """Compute Event pmaps and pmaps features.
    input:
    P           : CalibVectors named tuple.
                  ('CalibVectors',
                  'channel_id coeff_blr coeff_c adc_to_pes pmt_active')
    pmtrwf      : raw waveform for pmts
    sipmrwf     : raw waveform for SiPMs
    thr         : threshold for PMAP searches.
                  ('ThresholdParams',
                  'thr_s1 thr_s2 thr_MAU thr_sipm thr_SIPM')
    s12f        : named tuple containing features of S1 and S2
                  S12Features = namedtuple('S12Features',
                    's1f s1_pmt_f s1pf s2f ')
    event       : event number
    """

    RWF = pmtrwf[event]
    CWF                  = blr.deconv_pmt(RWF,
                                          P.coeff_c,
                                          P.coeff_blr,
                                          n_baseline  = deconv_n_baseline,
                                          thr_trigger = deconv_thr_trigger)

    # calibrated sum

    csum, csum_mau       = cpf.calibrated_pmt_sum(CWF,
                                                  P.adc_to_pes,
                                                  pmt_active   = P.pmt_active,
                                                  n_MAU        = 100,
                                                  thr_MAU      = thr.thr_MAU)


    CAL_PMT, CAL_PMT_MAU = cpf.calibrated_pmt_mau(CWF,
                                                  P.adc_to_pes,
                                                  pmt_active = P.pmt_active,
                                                  n_MAU      = 100,
                                                  thr_MAU    = thr.thr_MAU)


    s1_ene, s1_indx      = cpf.wfzs(              csum_mau,
                                                  threshold  =thr.thr_s1)
    S1                   =  cpf.find_S12(         s1_ene,
                                                  s1_indx,
                                                  **s1par._asdict())

    print_s12(S1)

    if len(S1) == 0:
        return EventPmaps.not_s1

    s2_ene, s2_indx = cpf.wfzs(csum, threshold=thr.thr_s1)
    S2    = cpf.find_S12(s2_ene, s2_indx, **s2par._asdict())

    print_s12(S2)
    if len(S2) == 0:
        return EventPmaps.not_s2

    for i in S1:
        print('S1: adding features for peak number {}'.format(i))
        s12f.s1f.add_features(self, S1, peak_number=i)

        # Search S1 for PMTs where we have found S1 for sum
        t = S1[i][0]
        tmin = t[0] - s1_match_pmt
        tmax = t[-1] + s1_match_pmt

        s1par_PMT = S12Params(tmin=tmin, tmax=tmax, lmin=3, lmax=20,
                              stride=4, rebin=False)

        PMT_S1 = {}
        for pmt in P.pmt_active:
            s1_ene, s1_indx = cpf.wfzs(CAL_PMT_MAU[pmt], threshold=0.1)
            PMT_S1[pmt] = cpf.find_S12(s1_ene, s1_indx, **s1par_PMT._asdict())
            s12f.s1_pmt_f.add_features(self, PMT_S1[pmt], peak_number=i)


        nm = compare_S1(S1, PMT_S1, peak=i)
        print('number of S1 - S1_PMT matches = {}'.format(nm))
        s12f.s1f.add_number_of_matches(nm)

    for i in S2:
        print('S2: adding features for peak number {}'.format(i))
        s12f.s2.add_features(S2, peak_number=i)

    # S1p
    t = S2[0][0]
    tmin = t[-1] + 5 * units.mus
    print('S1p search starts at {}'.format(tmin))

    s1p_params = S12Params(tmin=tmin, tmax=1300*units.mus, stride=4, lmin=4,
                           lmax=20, rebin=False)
    s1_ene, s1_indx = cpf.wfzs(csum_mau, threshold=0.1)
    S1p =  cpf.find_S12(s1_ene, s1_indx, **s1p_params._asdict())
    for i in S1p:
        s12f.s1pf.add_features(S1p, peak_number=i)


    if len(S1) != 1:
        return EventPmaps.s1_not_1

    if len(S2) != 1:
        return EventPmaps.s2_not_1  # TODO: treat the case with more than 1 s2


    dt = s2f.tpeak - s1f.tpeak

    print('drif time = {} mus'.format(dt/units.mus))

    print('***S2Si****')
    sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=3.5*units.pes, n_MAU=100)
    SIPM = cpf.select_sipm(sipm)
    S2Si = pf.sipm_s2_dict(SIPM, S2, thr=20*units.pes)

    return (CalibratedSum(csum=csum, csum_mau=csum_mau),
            PMaps(S1=S1, S2=S2, S2Si=S2Si))
